In [1]:
import numpy as np
import os

import sys
from pycox import datasets
from lifelines.datasets import load_rossi
from sksurv.datasets import (
    load_aids,
    load_breast_cancer,
    load_flchain,
    load_gbsg2,
    load_whas500,
)
from sklearn.preprocessing import LabelEncoder
import synthcity.logger as log
from medicaldata.CUTRACT import download as cutract_download, load as cutract_load
from medicaldata.SEER_prostate_cancer import download as seer_download, load as seer_load
from pathlib import Path

log.add(sink=sys.stderr, level="INFO")

def get_dataset(name: str):
    if name == "metabric":
        df = datasets.metabric.read_df()
    elif name == "support":
        df = datasets.support.read_df()
    elif name == "gbsg":
        df = datasets.gbsg.read_df()
    elif name == "rossi":
        df = load_rossi()
        df = df.rename(columns={"week": "duration", "arrest": "event"})
    elif name == "aids":
        X, Y = load_aids()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "flchain":
        X, Y = load_flchain()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "gbsg2":
        X, Y = load_gbsg2()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "whas500":
        X, Y = load_whas500()
        Y_unp = np.array(Y, dtype=[("event", "int"), ("duration", "float")])
        df = X.copy()
        df["event"] = Y_unp["event"]
        df["duration"] = Y_unp["duration"]
    elif name == "cutract":
        file_id = "1mew1S3-N2GdVu5nGjaqmpLo7sTKRf4Vj"
        csv_path = Path("data/cutract.csv")
        if not csv_path.exists():
            cutract_download(file_id, csv_path)
 
        X, T, Y = cutract_load(csv_path, preprocess = False)
        df = X.copy()
        df["event"] = Y
        df["duration"] = T
        
    elif name == "seer":
        file_id = "1PNXLjy8r1xHZq7SspduAMK6SGUTvuwM6"

        csv_path = Path("data/seer.csv")
        if not csv_path.exists():
            seer_download(file_id, csv_path)
 
        X, T, Y = seer_load(csv_path, preprocess = False)
        df = X.copy()
        df["event"] = Y
        df["duration"] = T
    for col in df.columns:
        if df[col].dtype.name in ["object", "category"]:
            df[col] = LabelEncoder().fit_transform(df[col])

    duration_col = "duration"
    event_col = "event"

    df = df.fillna(0)
    
    T = df[duration_col]
    
    time_horizons = np.linspace(T.min(), T.max(), num=5)[1:-1].tolist()

    return df, duration_col, event_col, time_horizons

In [2]:
df, duration_col, event_col, time_horizons = get_dataset("cutract")

df

,Age at diagnosis,PSA (ng/ml),Clinical T stage,Histological grade group,Comorbidity,Primary treatment type,event,duration
0,66,5,2,1.0,0.0,3,0,3353
1,72,0,1,1.0,0.0,1,0,5226
2,52,1,1,1.0,0.0,2,0,6028
3,68,1,1,1.0,0.0,3,0,4168
4,60,1,1,1.0,0.0,0,0,3407
...,...,...,...,...,...,...,...,...
10084,61,97,1,4.0,0.0,1,0,4091
10085,70,97,1,4.0,0.0,1,0,2399
10086,88,98,1,5.0,0.0,1,0,475
10087,57,99,1,5.0,1.0,0,0,2746


In [3]:
from synthcity.plugins import Plugins
from synthcity.benchmark import Benchmarks

plugins = Plugins().list(skip_debug = True)

plugins

['survival_tvae',
 'bayesian_network',
 'tvae',
 'survival_gan',
 'survival_bayesian_network',
 'survival_ctgan',
 'marginal_distributions',
 'gaussian_copula',
 'pategan',
 'survival_adsgan',
 'copulagan',
 'adsgan',
 'privbayes',
 'ctgan',
 'rtvae',
 'survival_nflow',
 'nflow']

In [4]:
base_plugins = [
    'privbayes',
    'adsgan',
    'bayesian_network',
    'ctgan',
    'tvae',
    'nflow',
]
survival_plugins = [
    'survival_gan',
    'survival_ctgan',
    'survival_tvae',
    'survival_bayesian_network',
    'survival_nflow',
]

repeats = 3

## AIDS dataset

In [5]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("aids")

df

,age,cd4,hemophil,ivdrug,karnof,priorzdv,raceth,sex,strat2,tx,txgrp,event,duration
0,34.0,169.0,0,0,0,39.0,0,0,1,0,0,0,189.0
1,34.0,149.5,0,0,3,15.0,1,1,1,0,0,0,287.0
2,20.0,23.5,1,0,0,9.0,0,0,0,1,1,0,242.0
3,48.0,46.0,0,0,3,53.0,0,0,1,0,0,0,199.0
4,46.0,10.0,0,2,3,12.0,0,0,0,1,1,0,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,44.0,65.5,0,0,0,103.0,0,0,1,1,1,0,273.0
1147,41.0,7.5,0,0,2,20.0,1,0,0,1,1,1,47.0
1148,43.0,170.0,0,2,3,27.0,1,0,1,0,0,0,272.0
1149,44.0,282.5,0,2,2,12.0,0,0,1,0,0,0,192.0


In [6]:
# base_plugins
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-05T16:28:41.908179+0300][149103][INFO] Benchmarking plugin : privbayes
[2022-05-05T16:28:41.909020+0300][149103][INFO]  Experiment repeat: 0
[2022-05-05T16:29:54.614909+0300][149103][INFO]  Experiment repeat: 1
[2022-05-05T16:31:07.109506+0300][149103][INFO]  Experiment repeat: 2
[2022-05-05T16:32:19.746392+0300][149103][INFO] Benchmarking plugin : adsgan
[2022-05-05T16:32:19.747069+0300][149103][INFO]  Experiment repeat: 0
[2022-05-05T16:33:09.050464+0300][149103][ERROR] Failed to evaluate synthetic performance. cox_ph: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
[2022-05-05T16:34:24.706129+0300][149103][INFO]  Experiment repeat: 1
[2022-05-05T16:36:19.645073+0300][149103][INFO]  Experiment repeat: 2
[2022-05-05T16:38:13.642945+0300][149103][INFO] Benchmarking plugin : bayesian_network
[2022-

In [7]:
Benchmarks.print(base_score)


Comparatives


,privbayes,adsgan,bayesian_network,ctgan,tvae,nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.046626,0.000000,0.000000,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.214315,0.075218,0.117401,0.087674,0.082663,0.080699
sanity.close_values_probability.score,0.488850,0.935708,0.889951,0.965827,0.924703,0.961483
sanity.distant_values_probability.score,0.002896,0.001738,0.001738,0.001738,0.002606,0.002027
stats.jensenshannon_dist.marginal,0.100467,0.097234,0.035120,0.121531,0.183452,0.063241
stats.chi_squared_test.marginal,0.741267,0.484030,0.761510,0.727763,0.222259,0.857147
stats.feature_corr.joint,2.019379,2.353104,0.732558,1.989845,2.055663,0.685895
stats.inv_kl_divergence.marginal,0.938704,0.862091,0.987808,0.930694,0.641119,0.971272
stats.ks_test.marginal,0.908641,0.907193,0.965225,0.866894,0.822028,0.931542


In [6]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-05T17:05:09.180484+0300][149654][INFO] Benchmarking plugin : survival_gan
[2022-05-05T17:05:09.181551+0300][149654][INFO]  Experiment repeat: 0
[2022-05-05T17:05:09.232854+0300][149654][INFO] Evaluate uncensoring for 7723803718165108694
[2022-05-05T17:05:09.267045+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.6942 duration = 0.0336 s
[2022-05-05T17:05:09.298748+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6966 duration = 0.0305 s
[2022-05-05T17:05:09.330335+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.7217 duration = 0.0306 s
[2022-05-05T17:07:15.312422+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.7313 duration = 125.9809 s
[2022-05-05T17:07:15.378974+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6996 duration = 0.0648 s
[2022-05-05T17:07:15.481948+0300][149654][INFO] [select_uncensori

[2022-05-05T17:46:19.581748+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.7324 duration = 0.083 s
[2022-05-05T17:46:19.660661+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6996 duration = 0.0776 s
[2022-05-05T17:46:19.777953+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.6793 duration = 0.1164 s
[2022-05-05T17:46:19.902135+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.6812 duration = 0.1236 s
[2022-05-05T17:46:48.769868+0300][149654][ERROR] Failed to evaluate synthetic performance. cox_ph: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
[2022-05-05T17:47:54.669770+0300][149654][INFO]  Experiment repeat: 1
[2022-05-05T17:47:54.743444+0300][149654][INFO] Evaluate uncensoring f

[2022-05-05T17:55:59.628581+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.6812 duration = 0.1495 s
[2022-05-05T17:59:03.359291+0300][149654][INFO]  Experiment repeat: 1
[2022-05-05T17:59:03.437430+0300][149654][INFO] Evaluate uncensoring for 7723803718165108694
[2022-05-05T17:59:03.480302+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.6942 duration = 0.0423 s
[2022-05-05T17:59:03.521374+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6966 duration = 0.0397 s
[2022-05-05T17:59:03.561718+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.7217 duration = 0.0392 s
[2022-05-05T17:59:03.657441+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.7324 duration = 0.0942 s
[2022-05-05T17:59:03.745023+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6996 duration = 0.0859 s
[2022-05-05T17:59:03.

In [7]:
Benchmarks.print(survival_score)


Comparatives


,survival_gan,survival_ctgan,survival_tvae,survival_bayesian_network,survival_nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.000000,0.020272,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.096125,0.155143,0.240465,0.125553,0.148962
sanity.close_values_probability.score,0.903852,0.706053,0.608746,0.812047,0.727483
sanity.distant_values_probability.score,0.002896,0.001448,0.040834,0.000869,0.001158
stats.jensenshannon_dist.marginal,0.146896,0.191798,0.252153,0.113011,0.140636
stats.chi_squared_test.marginal,0.530696,0.564287,0.172577,0.714620,0.803760
stats.feature_corr.joint,1.952260,2.137000,2.660606,1.431193,1.467282
stats.inv_kl_divergence.marginal,0.848522,0.833696,0.566244,0.882447,0.875515
stats.ks_test.marginal,0.840094,0.796364,0.747778,0.884515,0.853706


## FLChain dataset

In [ ]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("flchain")

df

In [9]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-05T18:05:19.725701+0300][149654][INFO] Benchmarking plugin : privbayes
[2022-05-05T18:05:19.726572+0300][149654][INFO]  Experiment repeat: 0
[2022-05-05T18:10:07.946582+0300][149654][INFO]  Experiment repeat: 1
[2022-05-05T18:14:48.305964+0300][149654][INFO]  Experiment repeat: 2
[2022-05-05T18:19:28.509663+0300][149654][INFO] Benchmarking plugin : adsgan
[2022-05-05T18:19:28.510291+0300][149654][INFO]  Experiment repeat: 0
[2022-05-05T18:24:22.742504+0300][149654][ERROR] Failed to evaluate synthetic performance. cox_ph: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
[2022-05-05T18:28:43.387681+0300][149654][INFO]  Experiment repeat: 1
[2022-05-05T18:33:28.986586+0300][149654][ERROR] Failed to evaluate synthetic performance. cox_ph: delta contains nan value(s). Convergence halted. Please see the 

In [10]:
Benchmarks.print(base_score)


Comparatives


,privbayes,adsgan,bayesian_network,ctgan,tvae,nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.067945,0.000000,0.000000,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.166375,0.079050,0.135732,0.165096,0.125240,0.145803
sanity.close_values_probability.score,0.724748,0.891542,0.773220,0.701380,0.828084,0.753662
sanity.distant_values_probability.score,0.000804,0.010795,0.000212,0.000296,0.000254,0.000847
stats.jensenshannon_dist.marginal,0.076220,0.151650,0.071674,0.085392,0.089198,0.079504
stats.chi_squared_test.marginal,0.814017,0.266023,0.722123,0.681102,0.559982,0.698676
stats.feature_corr.joint,2.865577,2.432189,0.960321,1.617332,1.236645,1.521469
stats.inv_kl_divergence.marginal,0.960031,0.757778,0.970838,0.959009,0.914574,0.953588
stats.ks_test.marginal,0.916965,0.874959,0.915764,0.907536,0.931312,0.905897


In [11]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-05T20:33:36.138329+0300][149654][INFO] Benchmarking plugin : survival_gan
[2022-05-05T20:33:36.139705+0300][149654][INFO]  Experiment repeat: 0
[2022-05-05T20:33:36.327245+0300][149654][INFO] Evaluate uncensoring for 412967762007448739
[2022-05-05T20:33:36.704579+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.0 duration = 0.3768 s
[2022-05-05T20:34:05.162657+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.9016 duration = 28.4575 s
[2022-05-05T20:36:41.922678+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.9318 duration = 156.7584 s
[2022-05-05T20:39:01.027199+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.9316 duration = 139.103 s
[2022-05-05T20:41:12.698991+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.9201 duration = 131.6713 s
[2022-05-05T20:43:43.715309+0300][149654][INFO] [select_uncensori

[2022-05-05T23:38:15.063174+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.9201 duration = 0.0683 s
[2022-05-05T23:41:15.656602+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.9299 duration = 180.5929 s
[2022-05-05T23:43:40.843169+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.9316 duration = 145.1858 s
[2022-05-05T23:50:44.241754+0300][149654][INFO]  Experiment repeat: 1
[2022-05-05T23:50:44.523661+0300][149654][INFO] Evaluate uncensoring for 412967762007448739
[2022-05-05T23:50:44.566739+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.0 duration = 0.0425 s
[2022-05-05T23:50:44.604090+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.9016 duration = 0.0368 s
[2022-05-05T23:50:44.631614+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.9318 duration = 0.0254 s
[2022-05-05T23:50:44.680130+0300]

[2022-05-06T02:52:40.056404+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.9317 duration = 835.2226 s
[2022-05-06T03:11:08.568143+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T03:11:08.850075+0300][149654][INFO] Evaluate uncensoring for 412967762007448739
[2022-05-06T03:11:08.894880+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.0 duration = 0.0443 s
[2022-05-06T03:11:08.939337+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.9016 duration = 0.0439 s
[2022-05-06T03:11:08.969898+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.9318 duration = 0.0289 s
[2022-05-06T03:11:09.024814+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.9321 duration = 0.0533 s
[2022-05-06T03:11:09.098765+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.9201 duration = 0.0725 s
[2022-05-06T03:14:58.41

In [12]:
Benchmarks.print(survival_score)


Comparatives


,survival_gan,survival_ctgan,survival_tvae,survival_bayesian_network,survival_nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.000000,0.067437,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.081765,0.034854,0.049015,0.047086,0.038348
sanity.close_values_probability.score,0.895098,0.989628,0.985734,0.989713,0.989671
sanity.distant_values_probability.score,0.001228,0.000593,0.000466,0.000381,0.000381
stats.jensenshannon_dist.marginal,0.156597,0.173548,0.164695,0.108046,0.166404
stats.chi_squared_test.marginal,0.340378,0.559643,0.433007,0.907900,0.527612
stats.feature_corr.joint,2.485735,2.524929,2.682427,2.086185,2.449146
stats.inv_kl_divergence.marginal,0.796807,0.836472,0.805683,0.884394,0.826758
stats.ks_test.marginal,0.826579,0.789518,0.833891,0.866257,0.789226


## gbsg2 dataset

In [13]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("gbsg2")

df

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize,event,duration
0,70.0,66.0,0,0,3.0,48.0,1,21.0,1,1814.0
1,56.0,77.0,1,0,7.0,61.0,1,12.0,1,2018.0
2,58.0,271.0,1,0,9.0,52.0,1,35.0,1,712.0
3,59.0,29.0,1,0,4.0,60.0,1,17.0,1,1807.0
4,73.0,65.0,0,0,1.0,26.0,1,35.0,1,772.0
5,32.0,13.0,0,1,24.0,0.0,2,57.0,1,448.0
6,59.0,0.0,1,0,2.0,181.0,1,8.0,0,2172.0
7,65.0,25.0,0,0,1.0,192.0,1,16.0,0,2161.0
8,80.0,59.0,0,0,30.0,0.0,1,39.0,1,471.0
9,66.0,3.0,0,0,7.0,0.0,1,18.0,0,2014.0


In [14]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T03:39:39.278907+0300][149654][INFO] Benchmarking plugin : privbayes
[2022-05-06T03:39:39.279728+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T03:40:42.362991+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T03:41:37.896775+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T03:42:33.089390+0300][149654][INFO] Benchmarking plugin : adsgan
[2022-05-06T03:42:33.089921+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T03:43:58.838168+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T03:45:28.098872+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T03:46:53.871086+0300][149654][INFO] Benchmarking plugin : bayesian_network
[2022-05-06T03:46:53.871703+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T03:47:50.868352+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T03:48:48.603825+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T03:49:44.878023+0300][149654][INFO] Benchmarking plugin : ctgan
[2022-05-06T03:49:44.878954+0300][149654][INFO]  E

In [15]:
Benchmarks.print(base_score)


Comparatives


,privbayes,adsgan,bayesian_network,ctgan,tvae,nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.326045,0.000000,0.000000,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.058633,0.046476,0.072696,0.097785,0.040169,0.048056
sanity.close_values_probability.score,0.983965,0.974247,0.904762,0.890185,0.967930,0.979592
sanity.distant_values_probability.score,0.002915,0.001458,0.004373,0.004859,0.001458,0.001944
stats.jensenshannon_dist.marginal,0.124579,0.115980,0.046117,0.142179,0.163154,0.074276
stats.chi_squared_test.marginal,0.610944,0.419693,0.811297,0.711559,0.450103,0.764880
stats.feature_corr.joint,1.905599,1.991500,0.646586,1.888074,0.851411,0.842858
stats.inv_kl_divergence.marginal,0.902445,0.859477,0.987159,0.876255,0.748663,0.963406
stats.ks_test.marginal,0.846064,0.850826,0.946647,0.819631,0.809378,0.890719


In [16]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T04:01:40.119512+0300][149654][INFO] Benchmarking plugin : survival_gan
[2022-05-06T04:01:40.121785+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T04:01:40.188726+0300][149654][INFO] Evaluate uncensoring for 2826583458290742066
[2022-05-06T04:01:47.079515+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.6705 duration = 6.89 s
[2022-05-06T04:01:49.051627+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6757 duration = 1.97 s
[2022-05-06T04:01:54.080579+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6932 duration = 5.0274 s
[2022-05-06T04:03:56.675024+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6936 duration = 122.5928 s
[2022-05-06T04:04:15.821338+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6332 duration = 19.1449 s
[2022-05-06T04:06:27.831221+0300][149654][INFO] [select_uncensoring_

[2022-05-06T04:39:40.295936+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6757 duration = 0.0321 s
[2022-05-06T04:39:40.333710+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6945 duration = 0.0364 s
[2022-05-06T04:39:40.400754+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6953 duration = 0.0655 s
[2022-05-06T04:39:40.494031+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6417 duration = 0.0914 s
[2022-05-06T04:39:40.597401+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.679 duration = 0.1029 s
[2022-05-06T04:39:40.722472+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.676 duration = 0.1245 s
[2022-05-06T04:40:51.820433+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T04:40:51.906326+0300][149654][INFO] Evaluate uncensoring for 2826583458290742066
[2022-05-06T04:40:51.944314+03

[2022-05-06T04:49:45.627939+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6953 duration = 0.0716 s
[2022-05-06T04:49:45.734915+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6417 duration = 0.1055 s
[2022-05-06T04:49:45.854266+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.679 duration = 0.1187 s
[2022-05-06T04:49:45.989438+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.676 duration = 0.1345 s


In [17]:
Benchmarks.print(survival_score)


Comparatives


,survival_gan,survival_ctgan,survival_tvae,survival_bayesian_network,survival_nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.000000,0.198251,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.060416,0.058052,0.051233,0.055004,0.050641
sanity.close_values_probability.score,0.944121,0.948008,0.949951,0.951409,0.950923
sanity.distant_values_probability.score,0.001458,0.001944,0.001458,0.002430,0.001458
stats.jensenshannon_dist.marginal,0.176509,0.185262,0.223999,0.110713,0.130995
stats.chi_squared_test.marginal,0.427170,0.506343,0.261285,0.726782,0.505334
stats.feature_corr.joint,1.926357,2.085575,1.561339,1.359012,1.475184
stats.inv_kl_divergence.marginal,0.748415,0.794937,0.600780,0.869583,0.830111
stats.ks_test.marginal,0.782702,0.791351,0.744072,0.884645,0.827600


## Metabric

In [18]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("gbsg2")

df

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize,event,duration
0,70.0,66.0,0,0,3.0,48.0,1,21.0,1,1814.0
1,56.0,77.0,1,0,7.0,61.0,1,12.0,1,2018.0
2,58.0,271.0,1,0,9.0,52.0,1,35.0,1,712.0
3,59.0,29.0,1,0,4.0,60.0,1,17.0,1,1807.0
4,73.0,65.0,0,0,1.0,26.0,1,35.0,1,772.0
5,32.0,13.0,0,1,24.0,0.0,2,57.0,1,448.0
6,59.0,0.0,1,0,2.0,181.0,1,8.0,0,2172.0
7,65.0,25.0,0,0,1.0,192.0,1,16.0,0,2161.0
8,80.0,59.0,0,0,30.0,0.0,1,39.0,1,471.0
9,66.0,3.0,0,0,7.0,0.0,1,18.0,0,2014.0


In [19]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T04:51:51.148309+0300][149654][INFO] Benchmarking plugin : privbayes
[2022-05-06T04:51:51.149291+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T04:52:46.642139+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T04:53:43.178158+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T04:54:39.273029+0300][149654][INFO] Benchmarking plugin : adsgan
[2022-05-06T04:54:39.273640+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T04:56:06.453627+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T04:57:33.581175+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T04:59:05.673321+0300][149654][INFO] Benchmarking plugin : bayesian_network
[2022-05-06T04:59:05.673848+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T05:00:01.920711+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T05:00:59.545909+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T05:01:55.663971+0300][149654][INFO] Benchmarking plugin : ctgan
[2022-05-06T05:01:55.664626+0300][149654][INFO]  E

In [20]:
Benchmarks.print(base_score)


Comparatives


,privbayes,adsgan,bayesian_network,ctgan,tvae,nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.326045,0.000000,0.000000,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.051029,0.041329,0.072696,0.097785,0.040169,0.048056
sanity.close_values_probability.score,0.987852,0.976676,0.904762,0.890185,0.967930,0.979592
sanity.distant_values_probability.score,0.001944,0.001458,0.004373,0.004859,0.001458,0.001944
stats.jensenshannon_dist.marginal,0.149839,0.105943,0.046117,0.142179,0.163154,0.074276
stats.chi_squared_test.marginal,0.693539,0.477369,0.811297,0.711559,0.450103,0.764880
stats.feature_corr.joint,1.940590,2.029936,0.646586,1.888074,0.851411,0.842858
stats.inv_kl_divergence.marginal,0.888368,0.867700,0.987159,0.876255,0.748663,0.963406
stats.ks_test.marginal,0.817736,0.860933,0.946647,0.819631,0.809378,0.890719


In [21]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T05:13:50.127491+0300][149654][INFO] Benchmarking plugin : survival_gan
[2022-05-06T05:13:50.128678+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T05:13:50.194218+0300][149654][INFO] Evaluate uncensoring for 2826583458290742066
[2022-05-06T05:13:50.237879+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.6705 duration = 0.043 s
[2022-05-06T05:13:50.281160+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6757 duration = 0.0416 s
[2022-05-06T05:13:50.325214+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6945 duration = 0.0426 s
[2022-05-06T05:13:50.400240+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6953 duration = 0.0735 s
[2022-05-06T05:13:50.504179+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6417 duration = 0.1021 s
[2022-05-06T05:13:50.619726+0300][149654][INFO] [select_uncensoring_

[2022-05-06T05:26:34.893642+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6757 duration = 0.044 s
[2022-05-06T05:26:34.938664+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6945 duration = 0.0435 s
[2022-05-06T05:26:35.022466+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6953 duration = 0.082 s
[2022-05-06T05:26:35.139400+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6417 duration = 0.1153 s
[2022-05-06T05:26:35.269668+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.679 duration = 0.1297 s
[2022-05-06T05:26:35.421521+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.676 duration = 0.1512 s
[2022-05-06T05:27:47.667545+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T05:27:47.754130+0300][149654][INFO] Evaluate uncensoring for 2826583458290742066
[2022-05-06T05:27:47.799889+0300

[2022-05-06T05:36:42.353474+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6953 duration = 0.0927 s
[2022-05-06T05:36:42.482350+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.6417 duration = 0.1276 s
[2022-05-06T05:36:42.626621+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.679 duration = 0.1436 s
[2022-05-06T05:36:42.804551+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.676 duration = 0.1773 s


In [22]:
Benchmarks.print(survival_score)


Comparatives


,survival_gan,survival_ctgan,survival_tvae,survival_bayesian_network,survival_nflow
sanity.data_mismatch.score,0.000000,0.000000,0.000000,0.000000,0.000000
sanity.common_rows_proportion.score,0.000000,0.000000,0.000000,0.206997,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.044860,0.068705,0.049064,0.056047,0.054618
sanity.close_values_probability.score,0.972789,0.947036,0.954810,0.947522,0.948008
sanity.distant_values_probability.score,0.001458,0.001458,0.001458,0.002430,0.001944
stats.jensenshannon_dist.marginal,0.163986,0.158986,0.220226,0.107102,0.123660
stats.chi_squared_test.marginal,0.538933,0.646622,0.253477,0.760024,0.512885
stats.feature_corr.joint,2.006991,2.126538,1.594859,1.363372,1.458574
stats.inv_kl_divergence.marginal,0.809310,0.839467,0.628950,0.875753,0.844589
stats.ks_test.marginal,0.794898,0.819485,0.754325,0.888192,0.836492


## gbsg

In [23]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("gbsg")

df

,x0,x1,x2,x3,x4,x5,x6,duration,event
0,0.0,0.0,0.0,32.0,1.0,155.0,168.0,84.000000,0
1,0.0,1.0,0.0,27.0,1.0,717.0,95.0,84.000000,0
2,0.0,1.0,1.0,52.0,1.0,120.0,437.0,84.000000,0
3,0.0,0.0,0.0,28.0,1.0,251.0,11.0,84.000000,0
4,0.0,0.0,0.0,39.0,1.0,241.0,92.0,66.234085,1
5,0.0,1.0,0.0,32.0,1.0,665.0,94.0,84.000000,0
6,0.0,0.0,1.0,56.0,1.0,110.0,238.0,84.000000,0
7,0.0,1.0,1.0,50.0,1.0,302.0,100.0,79.802872,1
8,0.0,0.0,1.0,59.0,1.0,523.0,962.0,84.000000,0
9,0.0,0.0,0.0,36.0,1.0,1506.0,85.0,84.000000,0


In [24]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T05:38:45.986473+0300][149654][INFO] Benchmarking plugin : privbayes
[2022-05-06T05:38:45.987388+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T05:40:42.162101+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T05:42:38.272585+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T05:44:44.830080+0300][149654][INFO] Benchmarking plugin : adsgan
[2022-05-06T05:44:44.830720+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T05:47:57.668678+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T05:51:09.734577+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T05:54:24.247685+0300][149654][INFO] Benchmarking plugin : bayesian_network
[2022-05-06T05:54:24.248293+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T05:56:29.614327+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T05:58:34.888335+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T06:00:49.975551+0300][149654][INFO] Benchmarking plugin : ctgan
[2022-05-06T06:00:49.976179+0300][149654][INFO]  E

In [25]:
Benchmarks.print(base_score)


Comparatives


,privbayes,adsgan,bayesian_network,ctgan,tvae,nflow
sanity.data_mismatch.score,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000
sanity.common_rows_proportion.score,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.011200,0.013527,0.014631,0.011087,0.009826,0.008695
sanity.close_values_probability.score,0.997312,0.990591,0.994922,0.996416,0.992533,0.997909
sanity.distant_values_probability.score,0.000597,0.000448,0.000448,0.000448,0.000448,0.000448
stats.jensenshannon_dist.marginal,0.099432,0.142348,0.057807,0.078218,0.099697,0.069700
stats.chi_squared_test.marginal,0.656124,0.516470,0.765264,0.749648,0.607624,0.742980
stats.feature_corr.joint,1.888362,2.051405,0.770052,1.329942,1.410482,0.708129
stats.inv_kl_divergence.marginal,0.933994,0.802113,0.966137,0.967978,0.889846,0.975431
stats.ks_test.marginal,0.867002,0.839274,0.932397,0.896721,0.894199,0.886134


In [26]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T06:30:04.315388+0300][149654][INFO] Benchmarking plugin : survival_gan
[2022-05-06T06:30:04.316373+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T06:30:04.404058+0300][149654][INFO] Evaluate uncensoring for 7818343538032827053
[2022-05-06T06:30:12.012166+0300][149654][INFO] [select_uncensoring_model] model = weibull_aft c_index_ood = 0.6627 duration = 7.6076 s
[2022-05-06T06:30:16.219220+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6643 duration = 4.2058 s
[2022-05-06T06:31:13.006529+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6717 duration = 56.7859 s
[2022-05-06T06:33:31.011382+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6856 duration = 138.0036 s
[2022-05-06T06:34:03.296581+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.5745 duration = 32.2838 s
[2022-05-06T06:36:52.544414+0300][149654][INFO] [select_uncenso

[2022-05-06T07:51:54.693000+0300][149654][INFO] [select_uncensoring_model] model = cox_ph c_index_ood = 0.6643 duration = 0.0365 s
[2022-05-06T07:51:54.717716+0300][149654][INFO] [select_uncensoring_model] model = random_survival_forest c_index_ood = 0.6717 duration = 0.0229 s
[2022-05-06T07:51:54.766687+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6856 duration = 0.0473 s
[2022-05-06T07:51:54.853660+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.634 duration = 0.0853 s
[2022-05-06T07:51:54.976930+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.6708 duration = 0.1227 s
[2022-05-06T07:54:37.318845+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.6681 duration = 162.3414 s
[2022-05-06T07:57:07.046558+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T07:57:07.217540+0300][149654][INFO] Evaluate uncensoring for 7818343538032827053
[2022-05-06T07:57:07.255148

[2022-05-06T08:35:06.665978+0300][149654][INFO] [select_uncensoring_model] model = survival_xgboost c_index_ood = 0.6856 duration = 0.0567 s
[2022-05-06T08:35:06.765587+0300][149654][INFO] [select_uncensoring_model] model = deephit c_index_ood = 0.634 duration = 0.098 s
[2022-05-06T08:35:06.909807+0300][149654][INFO] [select_uncensoring_model] model = tenn c_index_ood = 0.6708 duration = 0.1436 s
[2022-05-06T08:37:31.099177+0300][149654][INFO] [select_uncensoring_model] model = date c_index_ood = 0.6707 duration = 144.1888 s


In [27]:
Benchmarks.print(survival_score)


Comparatives


,survival_gan,survival_ctgan,survival_tvae,survival_bayesian_network,survival_nflow
sanity.data_mismatch.score,0.900000,0.900000,0.900000,0.900000,0.900000
sanity.common_rows_proportion.score,0.000000,0.000000,0.000000,0.197581,0.000000
sanity.nearest_syn_neighbor_distance.mean,0.008515,0.012023,0.009611,0.017701,0.009571
sanity.close_values_probability.score,0.992832,0.994624,0.992682,0.994624,0.997760
sanity.distant_values_probability.score,0.000448,0.000448,0.000448,0.000896,0.000448
stats.jensenshannon_dist.marginal,0.137135,0.140350,0.153728,0.090553,0.124543
stats.chi_squared_test.marginal,0.516357,0.517518,0.357178,0.745023,0.580696
stats.feature_corr.joint,1.712704,1.842520,1.933374,1.510513,1.538606
stats.inv_kl_divergence.marginal,0.804053,0.822646,0.743373,0.852817,0.822696
stats.ks_test.marginal,0.830247,0.843157,0.847919,0.914924,0.827459


## Support

In [ ]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("support")

df

In [ ]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


[2022-05-06T08:42:58.501017+0300][149654][INFO] Benchmarking plugin : privbayes
[2022-05-06T08:42:58.502153+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T08:49:17.916945+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T08:55:47.662578+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T09:02:18.615620+0300][149654][INFO] Benchmarking plugin : adsgan
[2022-05-06T09:02:18.616141+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T09:14:47.055261+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T09:27:11.377876+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T09:39:45.039739+0300][149654][INFO] Benchmarking plugin : bayesian_network
[2022-05-06T09:39:45.040653+0300][149654][INFO]  Experiment repeat: 0
[2022-05-06T10:00:28.752749+0300][149654][INFO]  Experiment repeat: 1
[2022-05-06T10:19:59.049067+0300][149654][INFO]  Experiment repeat: 2
[2022-05-06T10:40:52.834153+0300][149654][INFO] Benchmarking plugin : ctgan
[2022-05-06T10:40:52.834993+0300][149654][INFO]  E

In [ ]:
Benchmarks.print(base_score)

In [ ]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


In [ ]:
Benchmarks.print(survival_score)

## CUTRACT

In [ ]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("cutract")

df

In [ ]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


In [ ]:
Benchmarks.print(base_score)

In [ ]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


In [ ]:
Benchmarks.print(survival_score)

## SEER prostate 

In [ ]:
import pandas as pd

df, duration_col, event_col, time_horizons = get_dataset("seer")

df

In [ ]:
base_score = Benchmarks.evaluate(
    base_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


In [ ]:
Benchmarks.print(base_score)

In [ ]:
# survival plugins
survival_score = Benchmarks.evaluate(
    survival_plugins,
    df,
    task_type = "survival_analysis",
    target_column = event_col,
    time_to_event_column = duration_col,
    time_horizons = time_horizons,
    synthetic_size = len(df),
    repeats = repeats,
)


In [ ]:
Benchmarks.print(survival_score)